# Named Entity Recognition (NER)

Como hemos visto, un NER se encarga de etiquetar palabras en un texto en un conjunto de categorías (nombres de persona, de localización, de entidades de tiempo...).

<img src=https://miro.medium.com/max/1400/1*8LOMipM-fmszClg-AwATkQ.png width=600px>

A diferencia de un problema de clasificación de documentos (en el que todo el documento es clasificado en una categoría), aquí cada palabra (o, más correctamente, cada token) es etiquetado secuencialmente.

Existen distintas aproximaciones para abordar este tipo de problemas:
- **_Clásicas_**: principalmente basados en reglas
- **Machine Learning**: se distinguen a su vez dos categorías aquí:
    - **Clasificación multi-clase**: se busca clasificar cada token entre un conjunto de categorías sin información del contexto del token
    - **CRF**: modelar el texto secuencialmente mediante un grafo probabilístico. En cada token se extraen features que tratan de representar su contexto. [Aquí](https://www.depends-on-the-definition.com/named-entity-recognition-conditional-random-fields-python/) lo explican muy bien.
- **Deep learning**: estado del arte (como en casi todas las áreas). Utilizar modelos como las Bi-LSTM permite inferir el contexto de un token de una manera más completa tanto de tokens anteriores a el, como posteriores.

**Nosotros vamos a implementar una solución basada en un CRF**.

<img src=https://i.imgur.com/ukAr3Uh.jpg width=700px>


## Pasos que seguiremos para entrenar un modelo de reconocimiento de entidades

Etapa de **entrenamiento**
1. Obtener un corpus de entrenamiento
2. Etiquetar todos los tokens. Aquellos que no correspondan con una categoría se les asocia el label 'O'
3. Definir qué features se extraerán
4. Entrenar un modelo de clasificación secuencial para predecir las etiquetas de los tokens

Etapa de **testeo**
1. Obtener un corpus de test etiquetado
2. Lanzar el modelo etiquetando los tokens
3. Analizar resultados


## IO / IOB Encoding

Existen dos maneras principales de etiquetas las etiquetas (IO e IOB).

- **IO** (inside-ouside): cada token tendrá una etiqueta, **no tiene en cuenta entidades compuestas por varios tokens**, o chunks, como, por ejemplo, 'Nueva York'.
- **IOB** (inside-outside-beginning): cada token tendrá una etiqueta. Si tiene en cuenta chunks. Para codificar el inicio y final de las entidades se incluyen los prefijos 'B-' ('Beginning') e 'I-' ('Inside') para indicar que un token es el inicio de una entidad o pertenece a la misma.

En ambos, la etiqueta **'O'** ('Outside') significa que el token no pertenece a ninguna entidad.

<img src=https://image.slidesharecdn.com/07lectiener-160215132149/95/ie-named-entity-recognition-ner-36-638.jpg>

> Debate: ¿cuál pensáis que arroja mejores resultados?


## Features

Algunas de las features que podemos extraer son:
- **Palabras**
    - El token actual e información sobre el mismo (mayúsculas, signos de puntuación, ...)
    - Tokens anteriores / posteriores e información sobre ellos
    - Substrings (word shapes)
- **Información lingüística**
    - PoS tags (del token, de los anteriores / consecutivos)
- **Otras labels**
    - NER labels (del token actual, de los anteriores)


## ¡Al lío!

Vamos a entrenar nuestro primer modelos de reconocimiento de entidades. Para ello, utilizaremos:
- Dataset CoNLL 2002, con PoS Tags
- Librería sklearn_crf suite

# CoNLL 2002 Shared Tasks con PoS Tags

La CoNLL (Conference on Computational Natural Language Learning) es una conferencia anual organizada por la SIGNLL (ACL's Special Interest Group of Natural Language Processing). Usaremos un corpus con **frases en castellano** con información tanto de los **PoS Tags** como de **entidades etiquetadas**.

Links:
- https://www.plantl.gob.es/tecnologias-lenguaje/catalogo-TL/campanas-evaluacion/Paginas/conll-2002.aspx

# sklearn_crfsuite

Instalación: `pip install sklearn-crfsuite`

Links:
- Tutorial: https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html
- API reference: https://sklearn-crfsuite.readthedocs.io/en/latest/api.html

In [ ]:

!pip install unzip
!unzip CoNLL2002_NER.zip

In [ ]:
!pip install sklearn
#Modelo NER
!pip install sklearn-crfsuite

In [ ]:
import os
import io

import pandas as pd

import sklearn_crfsuite
!pip install numpy --upgrade
from sklearn_crfsuite import scorers
#Para poder evaluar el modelo:
from sklearn_crfsuite import metrics

# Funciones útiles

In [ ]:
def load_data(filepath):
    with io.open(filepath, encoding='latin-1') as f:
        idsent = 0
        sentences = []
        for l in f:
            if not len(l.strip()):
                sentence = []
                idsent += 1
            else:
                word, pos, ner = l.strip().split(' ')
                sentences.append(['sentence#'+str(idsent), word, pos, ner])

        df = pd.DataFrame(sentences)
        df.columns = ['Sentence#','word','pos','ner']

        return df

In [ ]:
def data_summary(data):
    print("Number of sentences: ", len(data.groupby(['Sentence#'])))

    words = list(set(data["word"].values))
    n_words = len(words)
    print("Number of words in the dataset: ", n_words)

    tags = list(set(data["ner"].values))
    print("NER Tags:", tags)
    n_tags = len(tags)
    print("Number of Labels: ", n_tags)

    print("What the dataset looks like:")
    display(data.head(10))
    return

In [ ]:
def get_sentences(data):
    sentgroups = data.groupby('Sentence#')

    sentences = []
    for name, g in sentgroups:
        s = []
        for row in g.itertuples():
            s.append((row.word,row.pos,row.ner))
        sentences.append(s)

    return sentences

In [ ]:
def get_labels(data):
    tags = list(set(data["ner"].values))
    return tags

# Carga de datos

In [ ]:
!pip install requirements.txt


In [ ]:
datasets_path = './CoNLL2002_NER'
corpus_train_file = 'esp.train'
corpus_test_file = 'esp.testa'
corpus_val_file = 'esp.testb'

In [ ]:
df_train = load_data(os.path.join(datasets_path, corpus_train_file))
df_test = load_data(os.path.join(datasets_path, corpus_test_file))
df_val = load_data(os.path.join(datasets_path, corpus_val_file))

In [ ]:
sentences_train = get_sentences(df_train)
sentences_test = get_sentences(df_test)
sentences_val = get_sentences(df_val)

In [ ]:
data_summary(df_train)

In [ ]:
data_summary(df_test)

In [ ]:
sentences_train[3]

In [ ]:
sentences_test[0]

# Extracción de features

Definimos las features que queremos usar.

In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2]
    }

    if i > 0:
        word_1 = sent[i-1][0]
        postag_1 = sent[i-1][1]

        features.update({
            '-1:word.lower()': word_1.lower(),
            '-1:word.istitle()': word_1.istitle(),
            '-1:word.isupper()': word_1.isupper(),
            '-1:postag': postag_1,
            '-1:postag[:2]': postag_1[:2]
        })
    else:
        features['BOS'] = True  # Beginning of sentence

    if i < len(sent)-1:
        word_1 = sent[i+1][0]
        postag_1 = sent[i+1][1]

        features.update({
            '+1:word.lower()': word_1.lower(),
            '+1:word.istitle()': word_1.istitle(),
            '+1:word.isupper()': word_1.isupper(),
            '+1:postag': postag_1,
            '+1:postag[:2]': postag_1[:2]
        })
    else:
        features['EOS'] = True  # End of sentence

    return features

In [ ]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [ ]:
def sent2labels(sent):
    return [word[2] for word in sent]

# Datos para entrenamiento, validación y testeo

In [ ]:
X_train = [sent2features(s) for s in sentences_train]
y_train = [sent2labels(s) for s in sentences_train]

X_test = [sent2features(s) for s in sentences_test]
y_test = [sent2labels(s) for s in sentences_test]

X_val = [sent2features(s) for s in sentences_val]
y_val = [sent2labels(s) for s in sentences_val]

In [ ]:
df_train.iloc[0]

In [ ]:
X_train[0][0]

In [ ]:
y_train[0][0]

# Entrenamiento

In [ ]:
import sklearn_crfsuite
crf_ = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,

)

try:
    crf_.fit(X_train, y_train)
except AttributeError:
    pass





In [ ]:
X_train[1]

# Evaluación

Emplear los datos de validación para _tunnear_ el modelo

In [ ]:
def eval(X, y, removeO=True):
    y_pred = crf_.predict(X)
    labels = list(crf_.classes_)

    if removeO:
        labels.remove('O')

    f1_score_ = metrics.flat_f1_score(y, y_pred, average='weighted', labels=labels)
    print('F1 score: {0:.3f}'.format(f1_score_))

    # Group B and I results
    sorted_labels = sorted(
        labels,
        key=lambda name: (name[1:], name[0])
    )
    print(sorted_labels)




In [ ]:
eval(X_val,y_val)

In [ ]:
eval(X_test,y_test)

# Resultados finales

Una vez ajustado el modelo, usar el test set para evaluar el modelo final.

In [ ]:
eval(X_test, y_test)

# Transiciones más probables / improbables

In [ ]:
from collections import Counter

In [ ]:
def print_transitions(transition_features):
    for (label_from, label_to), weight in transition_features:
        print('{0:10}->   {1:10}->   {2:10}'.format(label_from, label_to, weight))

In [ ]:
common_transitions = Counter(crf_.transition_features_).most_common(10)

print('Transiciones más probables')
print_transitions(common_transitions)

In [ ]:
uncommon_transitions = Counter(crf_.transition_features_).most_common()[-10:]

print('Transiciones más improbables')
print_transitions(uncommon_transitions)

# Características estado

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print('{0:10}->   {1:10}->   {2:10}'.format(weight, label, attr))

In [ ]:
positive_state_features = Counter(crf_.state_features_).most_common(10)

print('Las más positivas')
print_state_features(positive_state_features)

In [ ]:
negative_state_features = Counter(crf_.state_features_).most_common()[-10:]

print('Las más negativas')
print_state_features(negative_state_features)